In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
#import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#from google.colab.patches import cv2_imshow
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2 as cv 
import os
import json
import pickle
import tensorflow as tf
from tensorflow import keras

In [70]:
#path of dataset
path="E:/OpticalImages/UCF-101/"

In [71]:
label_list=[]
for file in os.listdir(path):
    filename= os.fsdecode(path+file)
    #print("Filename: ",filename)
    name=filename.split('/')[3]
    #print("Name: ", name)
    label_list.append(name)  
label={}#
idx=0
for i in label_list:
    label[i]=idx
    idx+=1
for i in label_list:
    print(i)
    print(label[i])

ApplyEyeMakeup
0
ApplyLipstick
1
Archery
2
BabyCrawling
3
BalanceBeam
4
BandMarching
5
BaseballPitch
6
Basketball
7
BasketballDunk
8
BenchPress
9
Biking
10
Billiards
11
BlowDryHair
12
BlowingCandles
13
BodyWeightSquats
14
Bowling
15
BoxingPunchingBag
16
BoxingSpeedBag
17
BreastStroke
18
BrushingTeeth
19
CleanAndJerk
20
CliffDiving
21
CricketBowling
22
CricketShot
23
CuttingInKitchen
24
Diving
25
Drumming
26
Fencing
27
FieldHockeyPenalty
28
FloorGymnastics
29
FrisbeeCatch
30
FrontCrawl
31
GolfSwing
32
Haircut
33
Hammering
34
HammerThrow
35
HandstandPushups
36
HandstandWalking
37
HeadMassage
38
HighJump
39
HorseRace
40
HorseRiding
41
HulaHoop
42
IceDancing
43
JavelinThrow
44
JugglingBalls
45
JumpingJack
46
JumpRope
47
Kayaking
48
Knitting
49
LongJump
50
Lunges
51
MilitaryParade
52
Mixing
53
MoppingFloor
54
Nunchucks
55
ParallelBars
56
PizzaTossing
57
PlayingCello
58
PlayingDaf
59
PlayingDhol
60
PlayingFlute
61
PlayingGuitar
62
PlayingPiano
63
PlayingSitar
64
PlayingTabla
65
PlayingViolin

In [86]:
labels=[]


In [90]:
def optical_image(folder_name):
    
    m=np.zeros((240,320, 3))
    images=[]
    count=0
    
    curr_folder=os.fsdecode(path+folder_name)
    
    Sum=np.zeros((240,320, 3))

    file_name=folder_name+".data"
    
    for file1 in os.listdir(curr_folder):
        if file1.endswith('.avi'):
            #print(file1)
            
            cap = cv.VideoCapture(curr_folder+'/' + file1)
            #print("cap: ",cap.isOpened())
            
            ret, first_frame = cap.read()
            frameRate = cap.get(5)
           
            prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
            
            mask = np.zeros_like(first_frame)
            
            mask[..., 1] = 255
           
            it=0
            while(cap.isOpened()):
                if(count==72):
                    break
                frameId = cap.get(1)
                ret, frame = cap.read()
                if (ret != True):
                    break
                gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
                flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
                mask[..., 0] = angle * 180 / np.pi / 2
                mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
                rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
                
                #Here rgb is optical flow of one frame
                rgb=np.array(rgb)
                rgb=np.float32(rgb)
                it+=1
                #print("shape: ", rgb.shape)
                
                Sum=rgb+Sum
                
                #print("Sum:", Sum)
                count=count+1
            
            #calculate average of 72 frame's OP_flow for each video
            avg=Sum/72.0
            #print(folder_name+"AVG",avg)
                
            #store all video's OP_Flow of each category(ApplyMakeUp) in images
            images.append(avg)
            
            #mapping each video's OP_flow with the corresponding category name
            labels.append(label[folder_name])
            #labels = np.asarray(labels)
            #print("labels",labels)
            
            cap.release()
            cv.destroyAllWindows()
            
    
    
    with open(file_name, 'wb') as f:
        pickle.dump(images, f)
    
   # print("count=",count)     
    
    

In [74]:
#creating list of each folder name and store in folder_list
folder_list=[]

dirs=os.listdir(path)

for file in dirs:
    filename=os.fsdecode(path+file)
    #print(filename)
    name=filename.split('/')[3]
   # optical_image2(name)
    print(name)
    folder_list.append(name)

ApplyEyeMakeup
ApplyLipstick
Archery
BabyCrawling
BalanceBeam
BandMarching
BaseballPitch
Basketball
BasketballDunk
BenchPress
Biking
Billiards
BlowDryHair
BlowingCandles
BodyWeightSquats
Bowling
BoxingPunchingBag
BoxingSpeedBag
BreastStroke
BrushingTeeth
CleanAndJerk
CliffDiving
CricketBowling
CricketShot
CuttingInKitchen
Diving
Drumming
Fencing
FieldHockeyPenalty
FloorGymnastics
FrisbeeCatch
FrontCrawl
GolfSwing
Haircut
Hammering
HammerThrow
HandstandPushups
HandstandWalking
HeadMassage
HighJump
HorseRace
HorseRiding
HulaHoop
IceDancing
JavelinThrow
JugglingBalls
JumpingJack
JumpRope
Kayaking
Knitting
LongJump
Lunges
MilitaryParade
Mixing
MoppingFloor
Nunchucks
ParallelBars
PizzaTossing
PlayingCello
PlayingDaf
PlayingDhol
PlayingFlute
PlayingGuitar
PlayingPiano
PlayingSitar
PlayingTabla
PlayingViolin
PoleVault
PullUps
Punch
PushUps
Rafting
RockClimbingIndoor
RopeClimbing
Rowing
SalsaSpin
ShavingBeard
Shotput
SkateBoarding
Skiing
Skijet
SkyDiving
SoccerJuggling
SoccerPenalty
StillRings

In [91]:
label_mapping=[]#mapping or labeling each frame with the video folder_name
_allShapeLable=np.zeros(())
for item in folder_list:
    print("----la la-----: ", item)
    
    optical_image(item)

    

----la la-----:  ApplyEyeMakeup
----la la-----:  ApplyLipstick
----la la-----:  Archery
----la la-----:  BabyCrawling
----la la-----:  BalanceBeam
----la la-----:  BandMarching
----la la-----:  BaseballPitch
----la la-----:  Basketball
----la la-----:  BasketballDunk
----la la-----:  BenchPress
----la la-----:  Biking
----la la-----:  Billiards
----la la-----:  BlowDryHair
----la la-----:  BlowingCandles
----la la-----:  BodyWeightSquats
----la la-----:  Bowling
----la la-----:  BoxingPunchingBag
----la la-----:  BoxingSpeedBag
----la la-----:  BreastStroke
----la la-----:  BrushingTeeth
----la la-----:  CleanAndJerk
----la la-----:  CliffDiving
----la la-----:  CricketBowling
----la la-----:  CricketShot
----la la-----:  CuttingInKitchen
----la la-----:  Diving
----la la-----:  Drumming
----la la-----:  Fencing
----la la-----:  FieldHockeyPenalty
----la la-----:  FloorGymnastics
----la la-----:  FrisbeeCatch
----la la-----:  FrontCrawl
----la la-----:  GolfSwing
----la la-----:  Hairc

In [98]:
lablePath='F:\Pose_Estimation_Based_Human_Activity_Recognition_Using_CNN-main/'

In [93]:
with open('Labeling.data', 'wb') as f:
    pickle.dump(labels, f)

In [116]:
with open(lablePath+'Labeling.data', 'rb') as l:
    lables=pickle.load(l)

In [100]:
)

(13197, 100)


In [107]:
imagePath="F:/Pose_Estimation_Based_Human_Activity_Recognition_Using_CNN-main/Optical_Image/"
merge=[]

file1='ApplyEyeMakeUp.data'
file2='ApplyLipstick.data'

with open(imagePath+file1, 'rb') as f:
    data_file1 = pickle.load(f)
    
data_file1=np.asarray(data_file1)
data_file1=np.float32(data_file1)

with open(imagePath+file2, 'rb') as f:
    data_file2 = pickle.load(f)
data_file2=np.asarray(data_file2)
data_file2=np.float32(data_file2)

_all=np.concatenate([data_file1, data_file2], axis=0)
_all=np.asarray(_all)
_all=np.float32(_all)

#merging all data file info into a list(_all)
for file in os.listdir(imagePath):
    if file.endswith('.data') and file!='ApplyEyeMakeup.data' and file!='ApplyLipstick.data':
        print("Start:---->  ", file)
        
        with open(imagePath+file, 'rb') as f:
            data_file = pickle.load(f)
            
        
        raw=np.asarray(data_file)
        raw=np.float32(raw)
        print('Single-Shape',raw.shape)
        
        _all=np.concatenate([_all, raw], axis=0)
        #print("dtype ",_all.dtype)
        
        _all=np.asarray(_all)
        
        print("AfterConcat --",_all.shape)
        print("End: ", file)
print("all_list shape:----> ", _all.shape)



Start:---->   Archery.data
Single-Shape (145, 240, 320, 3)
AfterConcat -- (404, 240, 320, 3)
End:  Archery.data
Start:---->   BabyCrawling.data
Single-Shape (132, 240, 320, 3)
AfterConcat -- (536, 240, 320, 3)
End:  BabyCrawling.data
Start:---->   BalanceBeam.data
Single-Shape (108, 240, 320, 3)
AfterConcat -- (644, 240, 320, 3)
End:  BalanceBeam.data
Start:---->   BandMarching.data
Single-Shape (155, 240, 320, 3)
AfterConcat -- (799, 240, 320, 3)
End:  BandMarching.data
Start:---->   BaseballPitch.data
Single-Shape (150, 240, 320, 3)
AfterConcat -- (949, 240, 320, 3)
End:  BaseballPitch.data
Start:---->   Basketball.data
Single-Shape (134, 240, 320, 3)
AfterConcat -- (1083, 240, 320, 3)
End:  Basketball.data
Start:---->   BasketballDunk.data
Single-Shape (131, 240, 320, 3)
AfterConcat -- (1214, 240, 320, 3)
End:  BasketballDunk.data
Start:---->   BenchPress.data
Single-Shape (160, 240, 320, 3)
AfterConcat -- (1374, 240, 320, 3)
End:  BenchPress.data
Start:---->   Biking.data
Single-Sh

Single-Shape (102, 240, 320, 3)
AfterConcat -- (9470, 240, 320, 3)
End:  PushUps.data
Start:---->   Rafting.data
Single-Shape (111, 240, 320, 3)
AfterConcat -- (9581, 240, 320, 3)
End:  Rafting.data
Start:---->   RockClimbingIndoor.data
Single-Shape (144, 240, 320, 3)
AfterConcat -- (9725, 240, 320, 3)
End:  RockClimbingIndoor.data
Start:---->   RopeClimbing.data
Single-Shape (119, 240, 320, 3)
AfterConcat -- (9844, 240, 320, 3)
End:  RopeClimbing.data
Start:---->   Rowing.data
Single-Shape (137, 240, 320, 3)
AfterConcat -- (9981, 240, 320, 3)
End:  Rowing.data
Start:---->   SalsaSpin.data
Single-Shape (133, 240, 320, 3)
AfterConcat -- (10114, 240, 320, 3)
End:  SalsaSpin.data
Start:---->   ShavingBeard.data
Single-Shape (161, 240, 320, 3)
AfterConcat -- (10275, 240, 320, 3)
End:  ShavingBeard.data
Start:---->   Shotput.data
Single-Shape (144, 240, 320, 3)
AfterConcat -- (10419, 240, 320, 3)
End:  Shotput.data
Start:---->   SkateBoarding.data
Single-Shape (120, 240, 320, 3)
AfterConcat

In [108]:
with open('ALL_OPTICAL_IMAGE.data', 'wb') as f:
        pickle.dump(_all, f)

In [109]:
with open('ALL_OPTICAL_IMAGE.data', 'rb') as f:
    images=pickle.load( f)
print(images.shape)

(13197, 240, 320, 3)


In [117]:
lables = np.asarray(lables)
lb = LabelBinarizer()
lables = lb.fit_transform(lables)
print(lables.shape)

(13197, 100)


In [118]:
print(images.shape)
print(lables.shape)

(13197, 240, 320, 3)
(13197, 100)


In [119]:
(trainX, testX, trainY, testY) = train_test_split(images, lables,
	test_size=0.25, stratify=lables, random_state=42)

In [122]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(240, 320, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

94773248/94765736 [==============================] - 117s 1us/step


In [128]:

print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / 10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(
	trainX, trainY, batch_size=48,
	steps_per_epoch=len(trainX) // 32,
	validation_data=(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=10)


[INFO] compiling model...
[INFO] training head...
Epoch 1/10
309/309 [==============================] - 1954s 6s/step - loss: 0.0427 - accuracy: 0.9980 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 2/10
309/309 [==============================] - 1728s 6s/step - loss: 0.0351 - accuracy: 0.9987
Epoch 3/10
309/309 [==============================] - 1748s 6s/step - loss: 0.0330 - accuracy: 0.9992
Epoch 4/10
309/309 [==============================] - 1732s 6s/step - loss: 0.0287 - accuracy: 0.9994
Epoch 5/10
309/309 [==============================] - 1680s 5s/step - loss: 0.0284 - accuracy: 0.9992
Epoch 6/10
309/309 [==============================] - 1578s 5s/step - loss: 0.0257 - accuracy: 0.9999
Epoch 7/10
309/309 [==============================] - 1097s 4s/step - loss: 0.0261 - accuracy: 0.9992


In [129]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX.astype("float32"), batch_size=48)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = 10
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

[INFO] evaluating network...


TypeError: object of type 'numpy.int32' has no len()

In [130]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 240, 320, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 326, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 120, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 120, 160, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [131]:
model.save('F:/CNN _Model/')

INFO:tensorflow:Assets written to: F:/CNN _Model/assets


In [2]:
model = tf.keras.models.load_model('F:/CNN _Model/')

# Check its architecture
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 240, 320, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 246, 326, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 120, 160, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 120, 160, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________